In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import mysql.connector

In [ ]:
def consultar_muestras_db(centroope, fecha_inicio, fecha_fin):
    """
    Consulta la base de datos para obtener los eventos de muestras filtrados por centroope y fechas.
    Retorna un DataFrame.
    """
    conexion = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD
    )
    
    query = f"""
    SELECT 
        e.idEvento, 
        e.fecha_evento, 
        e.id_evento_tipo, 
        e.coordenada_longitud, 
        e.coordenada_latitud, 
        e.medio_contacto, 
        e.tipo_evento, 
        e.id_categoria_evento, 
        bar.id AS id_barrio, 
        bar.barrio, 
        bar.id_estrato
    FROM 
        fullclean_contactos.vwEventos e
    LEFT JOIN 
        fullclean_contactos.vwContactos con ON e.id_contacto = con.id
    LEFT JOIN 
        fullclean_contactos.barrios bar ON bar.id = con.id_barrio
    LEFT JOIN 
        fullclean_contactos.ciudades ciu ON ciu.id = con.id_ciudad
    WHERE 
        e.fecha_evento BETWEEN '{fecha_inicio}' AND '{fecha_fin}'
        AND e.id_evento_tipo = 15
        AND ciu.id_centroope = {centroope}
        AND coordenada_longitud <> 0 
        AND coordenada_latitud <> 0;
    """
    df = pd.read_sql(query, conexion)
    conexion.close()
    return df